# POC SNE - Atendimento Serpro

## Instalação de pacotes necessários

In [1]:
# Importações básicas - Descomentar as linhas abaixo:
#!pip install pandas
#!pip install tensorflow
#!pip install keras
#!pip install scikit-learn
#!pip install nltk
#!pip install ezodf
#!pip install lxml
#import nltk
#nltk.download('all')
#!pip install spacy
#!python -m spacy download pt_core_news_sm

## Importações e criação de funções

In [2]:
import ezodf
import pandas as pd
import sys
import unicodedata
import re
import time
import spacy
from unicodedata import normalize
from nltk import tokenize # Tokenização
from nltk.corpus import stopwords # Stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import RSLPStemmer
# Função para mostrar as planilhas do documento
def show_ods_sheets(filename):
    doc = ezodf.opendoc(filename)
    print("Spreadsheet contains %d sheet(s)." % len(doc.sheets))
    for sheet in doc.sheets:
        print("-"*40)
        print("   Sheet name : '%s'" % sheet.name)
        print("Size of Sheet : (rows=%d, cols=%d)" % (sheet.nrows(), sheet.ncols()) )
# Função para converter ODS em DataFrame do Pandas
def read_sheet_ods_as_panda_dataframe(filename, sheet_no=0, header=0):
    tab = ezodf.opendoc(filename=filename).sheets[sheet_no]
    return pd.DataFrame({col[header].value:[x.value for x in col[header+1:]]
                         for col in tab.columns()})

## Abrindo o arquivo como um DataFrame

In [3]:
# Exibindo informações sobre o arquivo ODS
FILENAME = '/home/03662232677/workspace-neon/PUCOMEX/SS_SNE.ods'
show_ods_sheets(FILENAME)

Spreadsheet contains 1 sheet(s).
----------------------------------------
   Sheet name : 'Incidentes'
Size of Sheet : (rows=34120, cols=15)


In [4]:
# Lendo o arquivo ODS com o Pandas
df = read_sheet_ods_as_panda_dataframe(FILENAME)

## Inspecionando e trabalhando o DataFrame

In [5]:
# Mostrando os primeiros registros
df.head()

,ticketid,ownergroup,description,externalsystem,status,classificationid,serpronivelaten,descricaodemanda,solution,classificationid1,description2,relatedreckey,internalpriority,solucaoparausuario,None
0,2018SS/0000401275,ATCDMGSNE,SNE,EMAIL,CLOSED,10252,1NIVEL,Solicitação de serviço: 2018SS/0000401273 Qual...,,10252,SNE,,9,"Prezado Senhor André, O Sistema de Notificação...",None
1,2017SS/0000183234,UNCEDFSNE,SNE,EMAIL,CLOSED,10252,3NIVEL,Usuário informa que recebeu umas notificações ...,,10252,SNE,,9,"Sr. Usuário,\n\nO órgão autuador faz algumas v...",None
2,2017SS/0000183525,CAGSBRSNE,SNE,EMAIL,CLOSED,10252,2NIVEL,Usuário informa que emitiu um boleto que vence...,,10252,SNE,,9,"Prezado Angelo,\n\nQual a data de vencimento d...",None
3,2017SS/0000184325,CAGSBRSNE,SNE,EMAIL,CLOSED,10252,2NIVEL,Usuário informa que esta tentando cadastrar um...,,10252,SNE,,9,"Prezado Tiago,\nInformamos que o código de seg...",None
4,2017SS/0000242075,CAGSBRSNE,SNE,EMAIL,CLOSED,10252,2NIVEL,Assunto: MULTA PAGA VOLTANDO A CONTAR NO SISTE...,2016SOL/00400157,10252,SNE,,9,"Prezado Jeferson,\n\n\nFavor nos enviar PLACA ...",None


In [7]:
# Selecionando apenas as colunas desejadas
df = df[['description','ownergroup','descricaodemanda','solution','solucaoparausuario']]

In [8]:
# Mudando o nome das colunas
df.columns = ['descricao','grupo','descricao_demanda','solucao','solucao_usuario']
df.head()

,descricao,grupo,descricao_demanda,solucao,solucao_usuario
0,SNE,ATCDMGSNE,Solicitação de serviço: 2018SS/0000401273 Qual...,,"Prezado Senhor André, O Sistema de Notificação..."
1,SNE,UNCEDFSNE,Usuário informa que recebeu umas notificações ...,,"Sr. Usuário,\n\nO órgão autuador faz algumas v..."
2,SNE,CAGSBRSNE,Usuário informa que emitiu um boleto que vence...,,"Prezado Angelo,\n\nQual a data de vencimento d..."
3,SNE,CAGSBRSNE,Usuário informa que esta tentando cadastrar um...,,"Prezado Tiago,\nInformamos que o código de seg..."
4,SNE,CAGSBRSNE,Assunto: MULTA PAGA VOLTANDO A CONTAR NO SISTE...,2016SOL/00400157,"Prezado Jeferson,\n\n\nFavor nos enviar PLACA ..."


In [9]:
# Exibindo estatísticas do pandas
df.describe()

,descricao,grupo,descricao_demanda,solucao,solucao_usuario
count,34117,34117,34117,34117,34117
unique,49,23,32139,89,12451
top,"SNE,INFRAÇÃO / MULTA NÃO DISPONÍVEL NO SNE",ATCDMGSNE,,2017SOL/0000002593,"Prezado(a) Sr(a),\n\no cadastro pode ser feito..."
freq,5467,10463,161,5739,1119


In [10]:
# Removendo valores vazios em descricao_demanda
df = df.dropna(subset=['descricao_demanda']) 
df = df[df.descricao_demanda != '']
df.describe()

,descricao,grupo,descricao_demanda,solucao,solucao_usuario
count,33956,33956,33956,33956,33956
unique,48,23,32138,88,12367
top,"SNE,INFRAÇÃO / MULTA NÃO DISPONÍVEL NO SNE",ATCDMGSNE,Adesão ao SNE,2017SOL/0000002593,"Prezado(a) Sr(a),\n\no cadastro pode ser feito..."
freq,5447,10452,145,5686,1110


In [11]:
# removendo colunas indesejadas de grupo
df = df.dropna(subset=['grupo']) 
df = df[df.grupo != '']
grupos = df['grupo'].unique()
print(len(grupos))
grupos

22


array(['ATCDMGSNE', 'UNCEDFSNE', 'CAGSBRSNE', 'GSGSSPCENTRODECOMANDO',
       'ATGSRJ', 'ATGSPEDENATRAN', 'ATGSBRTERC', 'CACDBRLOJASSNE',
       'GSCDSPLINUX', 'CACDBRSNE', 'GSDEBAAPPSNE', 'GSCDSP',
       'GSGSDFCENTRODECOMANDO', 'ATGSMG', 'CAGSBRRENAVAM', 'CACDBRRADAR',
       'CACDBRCNHDIGITAL', 'CAGSBRRENAJUD', 'ATGSRJELETR', 'ATGSBREMAIL',
       'ATGSPE', 'GSSSDFRADAR'], dtype=object)

In [12]:
# Removendo quebra de linha da descricao
df['descricao'] = df['descricao'].replace(r'\n','', regex=True)
descricoes = df['descricao'].unique()
print(len(descricoes))
descricoes

48


array(['SNE', 'SNE,ACESSO', 'SNE,ACESSO,SISTEMA INDISPONÍVEL-LENTIDÃO',
       'SNE,ACESSO,SITIO INDISPONÍVEL- LENTIDÃO',
       'SNE,ACESSO,DÚVIDA - INFORMAÇÃO',
       'SNE,ACESSO,NAVEGADOR DE INTERNET', 'SNE,APLICAÇÃO',
       'SNE,APLICAÇÃO,ESCLARECIMENTO', 'SNE,CADASTRO',
       'SNE,CADASTRO,ACESSO', 'SNE,CADASTRO,DÚVIDA – INFORMAÇÃO',
       'SNE,CADASTRO,VEÍCULO', 'SNE,SOLICITAÇÃO',
       'SNE,DÚVIDA - INFORMAÇÃO', 'SNE,SOLICITAÇÃO,DÚVIDA - INFORMAÇÃO',
       'SNE,SITUAÇÃO NÃO PREVISTA', 'SNE,RECLAMACAO',
       'SNE,ERRO NA GERAÇÃO DE LINHA DIGITÁVEL/BOLETO DO DETRAN-DF',
       'SNE,SNE - ERRO NA GERAÇÃO DA LINHA DIGITÁVEL',
       'SNE,SNE - ERRO NO CADASTRO DO USUÁRIO NO SNE',
       'SNE,SNE - ERRO NO CADASTRO DE VEÍCULO NO SNE',
       'SNE,SNE - CADASTRO PESSOA JURÍDICA - CNPJ',
       'SNE,SNE - CADASTRO DE USUÁRIO PESSOA FÍSICA',
       'SNE,INFRAÇÃO / MULTA NÃO DISPONÍVEL NO SNE',
       'SNE,BAIXA DA MULTA NÃO DISPONÍVEL NO SNE',
       'SNE,SNE - MSG ERRO: OCORREU

In [14]:
# Extraindo o segundo e terceiro termo após a vírgula
import numpy as np
df['sistema'] =  (np.where(df['descricao'].str.contains(','),
                  df['descricao'].str.split(',').str[1],
                  df['descricao']))
df['classificacao'] =  (np.where(df['descricao'].str.contains(','),
                  df['descricao'].str.split(',').str[2],
                  df['descricao']))
# Reorganizando a ordem das colunas
df = df[['sistema','classificacao','grupo', 'descricao_demanda','descricao','solucao','solucao_usuario']]
# Removendo valores nulos das colunas criadas
df = df.dropna(subset=['sistema','classificacao'])
df = df[df.sistema != '']
df = df[df.classificacao != '']
df.describe()

,sistema,classificacao,grupo,descricao_demanda,descricao,solucao,solucao_usuario
count,6291,6291,6291,6291,6291,6291,6291
unique,7,19,20,5914,20,62,2056
top,SOLICITAÇÃO,DÚVIDA - INFORMAÇÃO,ATGSBRTERC,Usuário pergunta como pode se CADASTRAR no SIS...,"SNE,SOLICITAÇÃO,DÚVIDA - INFORMAÇÃO",,A consulta dos órgãos que fizeram adesão ao SN...
freq,1789,2041,2432,35,1789,1521,311


In [15]:
# Verificando quantos registros existem por sistema
df.groupby('sistema').count()

,classificacao,grupo,descricao_demanda,descricao,solucao,solucao_usuario
sistema,,,,,,
ACESSO,420,420,420,420,420,420
APLICAÇÃO,1438,1438,1438,1438,1438,1438
CADASTRO,1060,1060,1060,1060,1060,1060
SNE,572,572,572,572,572,572
SNE - ATENDIMENTO LOJAS,915,915,915,915,915,915
SNE - MSG ERRO: OCORREU UM ERRO INESPERADO,97,97,97,97,97,97
SOLICITAÇÃO,1789,1789,1789,1789,1789,1789


In [26]:
# Escolhendo um sistema para a POC
#SISTEMA_SELECIONADO = 'DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO'
#df = df.loc[df['sistema'] == SISTEMA_SELECIONADO]
# Mostrando quandos existem por classificacao
df.groupby('classificacao').count()

,sistema,grupo,descricao_demanda,descricao,solucao,solucao_usuario,descricao_demanda2,solucao_usuario2
classificacao,,,,,,,,
RETORNO INVÁLIDO DO SERVIDOR,97,97,97,97,97,97,97,97
ACESSO,13,13,13,13,13,13,13,13
DÚVIDA - INFORMAÇÃO,2041,2041,2041,2041,2041,2041,2041,2041
DÚVIDA – INFORMAÇÃO,968,968,968,968,968,968,968,968
ESCLARECIMENTO,1438,1438,1438,1438,1438,1438,1438,1438
NAVEGADOR DE INTERNET,53,53,53,53,53,53,53,53
SISTEMA INDISPONÍVEL-LENTIDÃO,100,100,100,100,100,100,100,100
SITIO INDISPONÍVEL- LENTIDÃO,15,15,15,15,15,15,15,15
SNE,572,572,572,572,572,572,572,572


In [15]:
# Removendo classificaoes com menos que X excemplos
MINIMO_REGISTROS = 50
counts = df['classificacao'].value_counts()
df = df[df['classificacao'].isin(counts[counts > MINIMO_REGISTROS].index)]
df.groupby('classificacao').count()

,sistema,grupo,descricao_demanda,descricao,solucao,solucao_usuario,solucao_script
classificacao,,,,,,,
AMBIENTE,362,362,362,362,362,362,362
ANÁLISE URC,130,130,130,130,130,130,130
APLICAÇÃO,471,471,471,471,471,471,471
DÚVIDA OU INFORMAÇÕES,1258,1258,1258,1258,1258,1258,1258
REGRA DE NEGÓCIO,81,81,81,81,81,81,81
SITUAÇÃO NÃO PREVISTA,4071,4071,4071,4071,4071,4071,4071


In [16]:
df2 = df[df.classificacao == 'SITUAÇÃO NÃO PREVISTA']
df = df[df.classificacao != 'SITUAÇÃO NÃO PREVISTA']
df.groupby('classificacao').count()

,sistema,grupo,descricao_demanda,descricao,solucao,solucao_usuario,solucao_script
classificacao,,,,,,,
AMBIENTE,362,362,362,362,362,362,362
ANÁLISE URC,130,130,130,130,130,130,130
APLICAÇÃO,471,471,471,471,471,471,471
DÚVIDA OU INFORMAÇÕES,1258,1258,1258,1258,1258,1258,1258
REGRA DE NEGÓCIO,81,81,81,81,81,81,81


In [17]:
df2.groupby('classificacao').count()

,sistema,grupo,descricao_demanda,descricao,solucao,solucao_usuario,solucao_script
classificacao,,,,,,,
SITUAÇÃO NÃO PREVISTA,4071,4071,4071,4071,4071,4071,4071


In [18]:
classificacoes = df['classificacao'].unique()
print(len(classificacoes))
classificacoes

5


array(['REGRA DE NEGÓCIO', 'ANÁLISE URC', 'DÚVIDA OU INFORMAÇÕES',
       'AMBIENTE', 'APLICAÇÃO'], dtype=object)

In [19]:
df.head()

,sistema,classificacao,grupo,descricao_demanda,descricao,solucao,solucao_usuario,solucao_script
0,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,REGRA DE NEGÓCIO,UNACDFPUCOMEX,"Bom Dia Tenho dois processos o qual ja voo , n...","CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",,Prezado Usuário\n\nAs cargas estão com a Améri...,#N/DISP
3,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,REGRA DE NEGÓCIO,UNACDFPUCOMEX,"Srs., Fizemos várias tentativas para obter inf...","CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",2017SOL/00003655,"Prezado Usuário,\n\nVocê pode estar tirando as...","\nSr(a). usuário(a),Sua solicitação foi atend..."
5,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,ANÁLISE URC,UNACDFPUCOMEX,"Recepcionamos a DUE 17BR0000257710, porem nao ...","CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",,"Caro Usuário,\n\nFavor informar a chave da not...",#N/DISP
9,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,ANÁLISE URC,UNACDFPUCOMEX,A cia aérea não consegue averbar a DUE 17BR000...,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",2017SOL/0000002832,"\nPrezado(a) Usuário(a),\n\nA DUE encontra-se ...","Sr(a). Usuário(a),É necessário, para análise d..."
11,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,REGRA DE NEGÓCIO,UNACDFPUCOMEX,GOSTARIA DE DETALHES DO MOTIVO DA NÃO AVERBAÇÃ...,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",2016SOL/00397182,"Prezado Usuário,\n\nO manifestador precisa ret...","Prezado(a) usuário(a),conforme informado atrav..."


## Preparação dos dados

In [16]:
# Estrutura para armazenar pontuações
tbl = dict.fromkeys(i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P'))
# Função para remover pontuações das sentenças
def remove_punctuation(text):
    return text.translate(tbl)

In [17]:
# Lista de palavras para inluir com regex
INCLUDE_WORDS_BY_REGEX = [r"^[A-Za-zçãàáâéêíóôõúüÂÃÁÀÉÊÍÓÔÕÚÜÇ'º°ª\.]*$",r"^[0-9\-/\\\.]*$"]
def include_words_by_regex(words):
    # Remove regex words
    words_aux = []
    for word in words:
        add = False
        for regex_exp in INCLUDE_WORDS_BY_REGEX:
            if re.compile(regex_exp).match(word):
                add = True
        if add:
            words_aux.append(word)
    return words_aux       
# Padroes regex para serem removidos do texto original
REMOVE_WORDS_BY_REGEX = [r"\S*@\S*\s?", # Email
    r"(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/[a-z0-9]*)*(\/)?", # Site
    r"([a-z0-9]+(\/)[a-z0-9]+)", # Protocolo
    r"(\(?\d{2,4}\)?)??[\-\s\.]?(\d{3,5})[\.\-]\d{4,5}", # Telefone
    r"\d{2}[\-\s\.]\d{4}", # Ramal
    r"([0-9]{2}[\.]?[0-9]{3}[\.]?[0-9]{3}[\/]?[0-9]{4}[-]?[0-9]{2})|([0-9]{3}[\.]?[0-9]{3}[\.]?[0-9]{3}[-]?[0-9]{2})", # CPF e CNPJ
    r"\d+" # Numeros
    #r"e-?mail:?" # texto email/e-mail
]
def remove_words_by_regex(text):
    for regex_exp in REMOVE_WORDS_BY_REGEX:
        text = re.sub(re.compile(regex_exp),"",text)
    return text
# Padroes regex para remoção do conteúdo a seguir
REMOVE_TEXT_AFTER_REGEX = [
        r"o acesso ao conte(u|ú)do desta mensagem",
        r"aviso legal",
        r"esta mensagem (e|é) destinada exclusivamente",
        r"legal advice",
        r"this message is exclusively destined"
]
def remove_text_after_regex(text):
    for regex_exp in REMOVE_TEXT_AFTER_REGEX:
        m = re.search(regex_exp, text)
        if m:
            text = text[:m.start()]
    return text

In [19]:
text = """"
srs., fizemos várias tentativas para obter informações em relação à adesão 
ao portal due, alguns exemplos: primeiramente acessamos o site: 
http://www.serpro.gov.br/menu/contato/cliente/ depois por telefone: 
0800-728-2323 e também nos contatos: 
sccd@serpro.gov.br<mailto:sccd@serpro.gov.br> 
http://comexresponde.comexbrasil.gov.br/ - protocolo: 30696 em uma das 
orientações recebidas (protocolo nº 2017ss/0000973710), nos informaram que 
o acesso seria através do mdic / comex, procedemos de acordo, porém não 
houve retorno. por favor, precisamos do contato urgente para alinhamento. 
favor contatar: fernanda cezar no telefone:3646-2256 / e-mail: 
fernanda.cezar@bradesco.com.br gratos. atenciosamente. banco bradesco s/a 
4861 / droc – departamento de relacionamento operacional com clientes setor 
apoio a projetos patrícia vidal tel: (11) 2646-3320 r: 31-3320 
classificação: interna “o acesso ao conteúdo desta mensagem está 
autorizado, exclusivamente, a colaboradores da organização bradesco. a 
reprodução a pessoas não enquadradas deve ser autorizada pelo gestor da 
informação, para identificá-lo contate o remetente". 
[cid:image164146.png@3690fe11.49984c89] aviso legal
...esta mensagem é destinada exclusivamente para a(s) pessoa(s) a quem é 
dirigida, podendo conter informação confidencial e/ou legalmente 
privilegiada. se você não for destinatário desta mensagem, desde já fica 
notificado de abster-se a divulgar, copiar, distribuir, examinar ou, de 
qualquer forma, utilizar a informação contida nesta mensagem, por ser 
ilegal. caso você tenha recebido esta mensagem por engano, pedimos que nos 
retorne este e-mail, promovendo, desde logo, a eliminação do seu conteúdo 
em sua base de dados, registros ou sistema de controle. fica desprovida de 
eficácia e validade a mensagem que contiver vínculos obrigacionais, 
expedida por quem não detenha poderes de representação. legal advice
...this message is exclusively destined for the people to whom it is 
directed, and it can bear private and/or legally exceptional information. 
if you are not addressee of this message, since now you are advised to not 
release, copy, distribute, check or, otherwise, use the information 
contained in this message, because it is illegal. if you received this 
message by mistake, we ask you to return this email, making possible, as 
soon as possible, the elimination of its contents of your database, 
registrations or controls system. the message that bears any mandatory 
links, issued by someone who has no representation powers, shall be null or 
void. [screenshot 1 saved as attachment image1641461512409665054.png by 
e-mail listener.]
"""
text = remove_words_by_regex(text.lower())
print(text)
text2 = remove_text_after_regex(text)
print(text2)

"
srs., fizemos várias tentativas para obter informações em relação à adesão 
ao portal due, alguns exemplos: primeiramente acessamos o site: 
 depois por telefone: 
 e também nos contatos: 

 - protocolo:  em uma das 
orientações recebidas (protocolo nº ), nos informaram que 
o acesso seria através do mdic / comex, procedemos de acordo, porém não 
houve retorno. por favor, precisamos do contato urgente para alinhamento. 
favor contatar: fernanda cezar no telefone: / e-mail: 
gratos. atenciosamente. banco bradesco  
 / droc – departamento de relacionamento operacional com clientes setor 
apoio a projetos patrícia vidal tel:  r:  
classificação: interna “o acesso ao conteúdo desta mensagem está 
autorizado, exclusivamente, a colaboradores da organização bradesco. a 
reprodução a pessoas não enquadradas deve ser autorizada pelo gestor da 
informação, para identificá-lo contate o remetente". 
aviso legal
...esta mensagem é destinada exclusivamente para a(s) pessoa(s) a quem é 
dirigida, p

In [20]:
spacy_pln = spacy.load('pt_core_news_sm')
words = [token.lemma_ for token in spacy_pln(text2)]
print (words)

['"', '\n', 'sr', '.', ',', 'fazer', 'vários', 'tentativo', 'parir', 'obter', 'informação', 'em', 'relação', 'à', 'adesão', '\n', 'o', 'o', 'portal', 'due', ',', 'algum', 'exemplo', ':', 'primeiramente', 'acessamos', 'o', 'site', ':', '\n ', 'depois', 'por', 'telefonar', ':', '\n ', 'e', 'também', 'o', 'contatos', ':', '\n\n ', '-', 'protocolo', ':', ' ', 'em', 'umar', 'das', '\n', 'orientação', 'recebido', '(', 'protocolo', 'nº', ')', ',', 'o', 'informar', 'que', '\n', 'o', 'acesso', 'seriar', 'através', 'do', 'mdic', '/', 'comex', ',', 'proceder', 'de', 'acordar', ',', 'porém', 'não', '\n', 'haver', 'retornar', '.', 'por', 'favor', ',', 'precisar', 'do', 'contato', 'urgente', 'parir', 'alinhamento', '.', '\n', 'favor', 'contatar', ':', 'fernanda', 'cezar', 'o', 'telefonar', ':', '/', 'e', '-', 'mail', ':', '\n', 'grato', '.', 'atenciosamente', '.', 'banco', 'bradesco', ' \n ', '/', 'droc', '–', 'departamento', 'de', 'relacionamento', 'operacional', 'com', 'cliente', 'setor', '\n', 'a

In [21]:
spacy_pln = spacy.load('pt_core_news_sm')
# Função para realizar o pré-processamento do texto livre
# Inicializa o stemmer
#stemmer = LancasterStemmer() 
stemmer = RSLPStemmer()
# Stop words para o Português
portuguese_stops = set(stopwords.words('portuguese'))
portuguese_stops.update('é')
REMOVER = ['attach','email']
LIMITE_TAMANHO_PALAVRA = 2
# Dicionátio que armazena as pontuações a serem removidas
def preprocessing(content):
    # Converte para minúsculo
    content = content.lower()
    # Remove elementos do texto via REGEX
    content = remove_words_by_regex(content)
    content = remove_text_after_regex(content)
    #Remover pontuação
    content = remove_punctuation(content)
    # Tokenização
    words = tokenize.word_tokenize(content, language='portuguese')
    #words = [token.lemma_ for token in spacy_pln(content)]
    # Remover acentuação
    words = [normalize('NFKD', word).encode('ASCII', 'ignore').decode('ASCII') for word in words]
    # Remove stopwords
    words = [word for word in words if word.lower() not in portuguese_stops] 
    # Stemming de cada palavra
    words = [stemmer.stem(w) for w in words if w.strip()] 
    # Remove palavras menores que um limite
    words = [word for word in words if len(word) > LIMITE_TAMANHO_PALAVRA] 
    # Remove by regex
    words = include_words_by_regex(words)
    # Remove specific words
    words = [word for word in words if word not in REMOVER] 
    return ' '.join(words)

In [22]:
# Imprimindo alguns exemplos
for sample in df['descricao_demanda'].head(5):
    print('============================================================================')
    print(sample)
    print('----------------------------------------------------------------------------')
    print(preprocessing(sample))

Solicitação de serviço: 2018SS/0000401273 Qual o procedimento agora?
----------------------------------------------------------------------------
solicitaca serv proced agor
Usuário informa que recebeu umas notificações em seu aplicativo de seu 
veículo, mas ao clicar na opção de 40% aparece apenas de 20%, sendo que não 
tem  entrau com nenhum recurso ou processo administrativo junto ao mesmo.

Placa: jgq1740
Órgão autuador: DER
Data da notificação de autuação: 31/01/2017
----------------------------------------------------------------------------
usuari inform receb uma notificaco aplic veicul clic opca aparec apen send nao entrau nenhum recurs process administr junt plac jgq orga autu der dat notificaca autuaca
Usuário informa que emitiu um boleto que venceu o prazo, e acabou vendendo 
o carro, e a multa sumiu tanto para ele como para pessoa que comprou o 
carro, o mesmo solicita orientações, a actuação foi pela Polícia Rodoviária 
Federal.

 Placa: epf9115
 Órgão autuador: Policia r

In [23]:
# Aplicando o pre-processamento
print('Iniciando a preparação das colunas textuais...')
start_time = time.time()
df['descricao_demanda2'] = df['descricao_demanda'].apply(preprocessing)
print('Processou a coluna descricao_demanda2 em %.2f segundos' % (time.time() - start_time))
start_time = time.time()
df['solucao_usuario2'] = df['solucao_usuario'].apply(preprocessing)
print('Processou a coluna solucao_usuario2 em %.2f segundos' % (time.time() - start_time))
# Combinando a classificação e grupo
#df['classificacao_grupo'] = df['classificacao'].astype(str) + ' - ' + df['grupo'].astype(str)
df.head()

Iniciando a preparação das colunas textuais...
Processou a coluna descricao_demanda2 em 8.58 segundos
Processou a coluna solucao_usuario2 em 15.31 segundos


,sistema,classificacao,grupo,descricao_demanda,descricao,solucao,solucao_usuario,descricao_demanda2,solucao_usuario2
0,SNE,SNE,ATCDMGSNE,Solicitação de serviço: 2018SS/0000401273 Qual...,SNE,,"Prezado Senhor André, O Sistema de Notificação...",solicitaca serv proced agor,prez senh andr sistem notificaca eletron sne d...
1,SNE,SNE,UNCEDFSNE,Usuário informa que recebeu umas notificações ...,SNE,,"Sr. Usuário,\n\nO órgão autuador faz algumas v...",usuari inform receb uma notificaco aplic veicu...,usuari orga autu faz algum verificaco sab real...
2,SNE,SNE,CAGSBRSNE,Usuário informa que emitiu um boleto que vence...,SNE,,"Prezado Angelo,\n\nQual a data de vencimento d...",usuari inform emit bolet venc praz acab vend c...,prez angel dat venc infraca comunidad atend sne
3,SNE,SNE,CAGSBRSNE,Usuário informa que esta tentando cadastrar um...,SNE,,"Prezado Tiago,\nInformamos que o código de seg...",usuari inform tent cadastr veicul sne inser co...,prez tiag inform codig seguranc crvdut encontr...
4,SNE,SNE,CAGSBRSNE,Assunto: MULTA PAGA VOLTANDO A CONTAR NO SISTE...,SNE,2016SOL/00400157,"Prezado Jeferson,\n\n\nFavor nos enviar PLACA ...",assunt mult pag volt cont sistem sne jeferson ...,prez jeferson favor envi plac ren poss verific...


In [24]:
df.describe()

,sistema,classificacao,grupo,descricao_demanda,descricao,solucao,solucao_usuario,descricao_demanda2,solucao_usuario2
count,6291,6291,6291,6291,6291,6291,6291,6291,6291
unique,7,19,20,5914,20,62,2056,5598,1944
top,SOLICITAÇÃO,DÚVIDA - INFORMAÇÃO,ATGSBRTERC,Usuário pergunta como pode se CADASTRAR no SIS...,"SNE,SOLICITAÇÃO,DÚVIDA - INFORMAÇÃO",,A consulta dos órgãos que fizeram adesão ao SN...,usuari pergunt pod cadastr sistem,prezado sra realiz cadastr soluca sra pod baix...
freq,1789,2041,2432,35,1789,1521,311,35,353


In [25]:
FILENAME = 'dataframe.pickle'
df.to_pickle(FILENAME)
print('arquivo salvo com sucesso!')

arquivo salvo com sucesso!


FIM

In [33]:
# Aplicando o pre-processamento
print('Iniciando a preparação das colunas textuais...')
start_time = time.time()
df2['descricao_demanda2'] = df2['descricao_demanda'].apply(preprocessing)
print('Processou a coluna descricao_demanda2 em %.2f segundos' % (time.time() - start_time))
start_time = time.time()
df2['solucao_usuario2'] = df2['solucao_usuario'].apply(preprocessing)
print('Processou a coluna solucao_usuario2 em %.2f segundos' % (time.time() - start_time))
start_time = time.time()
df2['solucao_script2'] = df2['solucao_script'].apply(preprocessing)
print('Processou a coluna solucao_script2 em %.2f segundos' % (time.time() - start_time))
# Combinando a classificação e grupo
df2['classificacao_grupo'] = df2['classificacao'].astype(str) + ' - ' + df2['grupo'].astype(str)
df2.head()

Iniciando a preparação das colunas textuais...
Processou a coluna descricao_demanda2 em 9.63 segundos
Processou a coluna solucao_usuario2 em 5.85 segundos
Processou a coluna solucao_script2 em 3.28 segundos


,sistema,classificacao,grupo,descricao_demanda,descricao,solucao,solucao_usuario,solucao_script,descricao_demanda2,solucao_usuario2,solucao_script2,classificacao_grupo
10,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CAGSBRPUCOMEX,"Bom dia, ao tentarmos retificar a Due no porta...","CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",2017SOL/0000002898,Favor solicitar a\nreexecução da funcionalidad...,"Sr(a). Usuário(a),Para análise da situação, é ...",bom dia tent retific due port receb err inform...,favor solic reexecuca funcional poi ambi dispo...,sra usuarioap analis situaca necessari acess p...,SITUAÇÃO NÃO PREVISTA - CAGSBRPUCOMEX
12,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CAGSBRPUCOMEX,"Sem acesso a criação de DU-E, opção de Embarqu...","CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",2016SOL/00397102,"Prezado usuario,segue a informação do nosso de...","Sr(a). usuário(a), conforme informado através...",acess criaca due opca embarqu antecip vinculac...,prez usuarioseg informaca desenvolv problem an...,sra usuario conform inform atrav contat telefo...,SITUAÇÃO NÃO PREVISTA - CAGSBRPUCOMEX
23,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CAGSBRPUCOMEX,"Bom dia, Conforme contato telefônico no qual f...","CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",2017SOL/00003655,"Prezado Edson,\nBom dia!\nA versão mais nova d...","\nSr(a). usuário(a),Sua solicitação foi atend...",bom dia conform contat telefon ger protocol se...,prez edson bom dia versa nov due produca hoj d...,sra usuarioasu solicitaca iosamentecentr serv rfb,SITUAÇÃO NÃO PREVISTA - CAGSBRPUCOMEX
26,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CAGSBRPUCOMEX,O sistema está passando por uma indisponibilid...,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",2017SOL/0000006065,"Prezado Rafael,\nBoa tarde!\nConforme contato,...",p {\nmargin-bottom: 0.21cm;\n}\na:link {\n}\n\...,sistem pass indisponibil momentane possi agu a...,prez rafael boa tard conform contat document c...,marginbottom alink sra usuarioasu solicitaca i...,SITUAÇÃO NÃO PREVISTA - CAGSBRPUCOMEX
27,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CAGSBRNOVOEX,Erro ao consultar DUE,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",2016SOL/00400731,\n\n,"Sr(a) Usuário(a), A fim de darmos continuidad...",err consult due,,sra usuario fim darm continu demand solicit ut...,SITUAÇÃO NÃO PREVISTA - CAGSBRNOVOEX


In [34]:
FILENAME2 = 'dataframe2.pickle'
df2.to_pickle(FILENAME2)
print('arquivo salvo com sucesso!')

arquivo salvo com sucesso!
